# Grand canonical ensemble transition-matrix Monte Carlo

In this example, flat histogram methods are employed for a small macrostate range from 0 to 5 particles.
To begin, the system is initialized with the minimum number of particles by setting Metropolis acceptance criteria with favorable conditions for adding particles.
The Metropolis criteria are then replaced with the flat histogram criteria.
At this point, typical analysis from the previous tutorials are added.
In addition, we also add checkpoint files, criteria status, and average energy of a given macrostate.
Finally, the simulation is run until the requested number of cycles of the flat histogram algorithm are complete.

A small macrostate range allows the simulation to run quickly with good sampling, and thus it is an ideal starting point to test the simulations. To begin, read the previous SRSW values from file for comparison.

In [1]:
import pandas as pd
import numpy as np

srsw_file = "../test/data/stat150.csv"
fh=pd.read_csv(srsw_file)[:6]
fh['lnPI'] -= np.log(sum(np.exp(fh['lnPI'])))   # renormalize
fh['delta_ln_prob']=fh['lnPI'].diff()
fh

,N,energy,lnPI,energystd,lnPIstd,delta_ln_prob
0,0,-2.312265e-10,-18.707570,6.689238e-10,0.037092,NaN
1,1,-6.057402e-04,-14.037373,6.709198e-10,0.036838,4.670197
2,2,-3.057422e-02,-10.050312,9.649147e-06,0.036964,3.987061
3,3,-8.992832e-02,-6.458921,1.387472e-04,0.037746,3.591391
4,4,-1.784571e-01,-3.145637,3.315245e-05,0.038097,3.313283
5,5,-2.961920e-01,-0.045677,1.348791e-05,0.038458,3.099960


Run a short flat-histogram simulation to compare with these values

In [2]:
script="""
MonteCarlo
Configuration cubic_side_length 8 particle_type0 /feasst/particle/lj.fstprt
Potential Model LennardJones
Potential VisitModel LongRangeCorrections
ThermoParams beta {beta} chemical_potential0 -2.352321
FlatHistogram Macrostate MacrostateNumParticles width 1 min 0 max 5 \
              Bias TransitionMatrix min_sweeps 50
TrialTranslate weight 0.25 tunable_param 1
TrialTransfer weight 1 particle_type 0
Log trials_per_write 1e5 output_file lj.txt
Movie trials_per_write 1e5 output_file lj.xyz
CheckEnergy trials_per_update 1e5 tolerance 1e-8
Tune
CriteriaUpdater trials_per_update 1e5
CriteriaWriter trials_per_write 1e5 output_file lj_fh.txt
Energy output_file lj_en.txt trials_per_update 1 trials_per_write 1e5 multistate true
Checkpoint checkpoint_file checkpoint.fst num_hours 0.0001
Run until complete
""".format(beta=1./1.5)

def run(script):
    with open('script1.txt', 'w') as file: file.write(script)
    import subprocess
    syscode = subprocess.call("../../../build/bin/fst < script1.txt > script1.log", shell=True, executable='/bin/bash')
    with open('script1.log', 'r') as file: print(file.read(), '\n', 'exit:', syscode)
run(script)

# Usage: ./fst < file.txt
FEASST version 0.25.6
MonteCarlo
Configuration cubic_side_length 8 particle_type0 /home/user/feasst/particle/lj.fstprt  
Potential Model LennardJones  
Potential VisitModel LongRangeCorrections  
ThermoParams beta 0.6666666666666666 chemical_potential0 -2.352321  
FlatHistogram Bias TransitionMatrix Macrostate MacrostateNumParticles max 5 min 0 min_sweeps 50 width 1  
TrialTranslate tunable_param 1 weight 0.25  
TrialTransfer particle_type 0 weight 1  
Log output_file lj.txt trials_per_write 1e5  
Movie output_file lj.xyz trials_per_write 1e5  
CheckEnergy tolerance 1e-8 trials_per_update 1e5  
Tune  
CriteriaUpdater trials_per_update 1e5  
CriteriaWriter output_file lj_fh.txt trials_per_write 1e5  
Energy multistate true output_file lj_en.txt trials_per_update 1 trials_per_write 1e5  
Checkpoint checkpoint_file checkpoint.fst num_hours 0.0001  
Run until complete  
# initializing random number generator with seed: 1734453337
 
 exit: 0


In [3]:
import pandas as pd

"""Compare the free energies and potential energies with the NIST SRSW
https://www.nist.gov/programs-projects/nist-standard-reference-simulation-website
https://mmlapps.nist.gov/srs/LJ_PURE/eostmmc.htm
"""
fh2=pd.read_csv('lj_fh.txt', comment='#')
en=pd.read_csv('lj_en.txt')
print(fh2[['delta_ln_prob', 'delta_ln_prob_stdev']])
print('energy', en[['average', 'block_stdev']])
for macro in range(1, len(fh2)):
    assert np.abs(fh["delta_ln_prob"][macro] - fh2["delta_ln_prob"][macro]) < 3*fh2["delta_ln_prob_stdev"][macro]
    assert np.abs(fh['energy'][macro] - en['average'][macro]) < 3*(fh["energystd"][macro]**2 + en['block_stdev'][macro]**2)**(1./2.)

   delta_ln_prob  delta_ln_prob_stdev
0            NaN             0.000000
1        4.67327             0.001854
2        3.98474             0.001643
3        3.58667             0.001978
4        3.31051             0.002104
5        3.10046             0.001927
energy         average   block_stdev
0  3.088249e-16  2.760036e-16
1 -6.057400e-04  7.702391e-10
2 -3.084554e-02  2.598340e-04
3 -8.976461e-02  4.648479e-04
4 -1.784771e-01  7.394942e-04
5 -2.983848e-01  1.408980e-03


A number of files should also have been created.
If the flat histogram method is sampling perfectly, the simulation performs a random walk along the macrostate.
For larger ranges of macrostates, or for more difficult sampling cases, monitoring the macrostate can help you determine what conditions are preventing convergence.
For example, a plot of the macrostate as a function of the number of attempts may look like the following:


In [4]:
pd.read_csv("lj.txt", header=0).dropna(axis='columns')

,volume,num_particles_of_type0,beta,state,energy,LennardJones,LongRangeCorrections,BondTwoBody,BondThreeBody,BondFourBody,trial,TrialTranslate,tunable,TrialAdd,TrialRemove
0,512,5,0.666667,5,-8.727640e-02,-7.213290e-02,-1.514350e-02,0,0,0,100000,0.924077,2.8307,0.043735,0.043833
1,512,5,0.666667,5,-6.660980e-01,-6.509540e-01,-1.514350e-02,0,0,0,200000,0.938036,2.8307,0.381398,0.409809
2,512,3,0.666667,3,-5.451660e-03,1.998400e-15,-5.451660e-03,0,0,0,300000,0.944572,2.8307,0.523987,0.583411
3,512,5,0.666667,5,-1.531790e-01,-1.380350e-01,-1.514350e-02,0,0,0,400000,0.948249,2.8307,0.595659,0.675027
4,512,2,0.666667,2,-2.422960e-03,4.718450e-16,-2.422960e-03,0,0,0,500000,0.950444,2.8307,0.638244,0.731855
5,512,5,0.666667,5,-3.239200e-02,-1.724850e-02,-1.514350e-02,0,0,0,600000,0.952298,2.8307,0.667888,0.771003
6,512,0,0.666667,0,1.253450e-15,-2.688820e-15,-1.626300e-18,0,0,0,700000,0.953237,2.8307,0.688676,0.798975
7,512,2,0.666667,2,-2.422960e-03,3.219650e-15,-2.422960e-03,0,0,0,800000,0.953807,2.8307,0.704320,0.820270
8,512,0,0.666667,0,-7.241390e-16,-2.220450e-16,-1.626300e-18,0,0,0,900000,0.954429,2.8307,0.716680,0.837076
9,512,3,0.666667,3,-1.120650e-02,-5.754840e-03,-5.451660e-03,0,0,0,1000000,0.954842,2.8307,0.725920,0.850215


Did this tutorial work as expected? Did you find any inconsistencies or have any comments? Please [contact](../../../CONTACT.rst) us. Any feedback is appreciated!